In [14]:
import pandas as pd
import numpy as np
import pingouin as pg
import numpy as np, statsmodels.stats.api as sms
import requests
from bs4 import BeautifulSoup 

with open('C:/Users/USER/Desktop/연구 결과물/청년가구/청년가구 gb AUC 부트스트랩 95% 신뢰 구간.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

In [15]:
from selenium import webdriver

In [16]:
options = webdriver.ChromeOptions() 
options.add_argument('lang=ko_KR') 
options.add_argument('disable-gpu')
driver = webdriver.Chrome(options=options)

### XGBClassifier

In [17]:
driver.get(url='.html')

In [18]:
df = pd.DataFrame()

In [19]:
import warnings
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [20]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [21]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')).any()]
df = df.drop(columns=cols_to_drop)

In [22]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [23]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [24]:
df

,Column_27,Column_44,Column_59,Column_73,Column_86,Column_98,Column_111,Column_124,Column_138,Column_152,...,Column_635,Column_648,Column_661,Column_674,Column_687,Column_700,Column_713,Column_726,Column_739,Column_752
0,"0.8492802337671651, 0.8510648596321393, 0.8534...","0.8513570685884335, 0.8490884156179413, 0.8479...","0.8517129181456383, 0.8526361550320893, 0.8533...","0.8480840593739916, 0.852914470617762, 0.85158...","0.8508013337635797, 0.8522699974902298, 0.8515...","0.8507390376824067, 0.8521790183213224, 0.8527...","0.8482808074289196, 0.8499014018859129, 0.8506...","0.8507457602810942, 0.8513364526191245, 0.8508...","0.8491556416048188, 0.8504105266931986, 0.8542...","0.8504176974651321, 0.849522247319924, 0.85060...",...,"0.8470227851278191, 0.8479012046896848, 0.8498...","0.847414040371446, 0.8453851600874834, 0.84655...","0.8500156860636047, 0.8488898748700298, 0.8486...","0.8489369330608441, 0.8492793374206734, 0.8480...","0.8479280950844358, 0.8442230468609946, 0.8492...","0.8453681295041411, 0.8447819188985695, 0.8460...","0.8468202108206948, 0.8431940410885231, 0.8441...","0.8393065863540209, 0.8379638593094546, 0.8378...","0.8244725000896347, 0.8250273385679967, 0.8248...","0.7780090351726363, 0.7780090351726363, 0.7780..."


In [25]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'xgb_{i}' for i in range(len(df.columns))])

In [26]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data)

112000

In [27]:
pd.set_option('display.precision', 16)

In [28]:
new_df.shape

(2000, 56)

In [29]:
new_df = new_df.astype(float)

In [30]:
new_df

,xgb_0,xgb_1,xgb_2,xgb_3,xgb_4,xgb_5,xgb_6,xgb_7,xgb_8,xgb_9,...,xgb_46,xgb_47,xgb_48,xgb_49,xgb_50,xgb_51,xgb_52,xgb_53,xgb_54,xgb_55
0,0.8492802337671651,0.8513570685884335,0.8517129181456383,0.8480840593739916,0.8508013337635797,0.8507390376824067,0.8482808074289196,0.8507457602810942,0.8491556416048188,0.8504176974651321,...,0.8470227851278191,0.8474140403714460,0.8500156860636047,0.8489369330608441,0.8479280950844358,0.8453681295041411,0.8468202108206948,0.8393065863540209,0.8244725000896347,0.7780090351726363
1,0.8510648596321393,0.8490884156179413,0.8526361550320893,0.8529144706177620,0.8522699974902298,0.8521790183213224,0.8499014018859129,0.8513364526191245,0.8504105266931986,0.8495222473199240,...,0.8479012046896848,0.8453851600874834,0.8488898748700298,0.8492793374206734,0.8442230468609946,0.8447819188985695,0.8431940410885231,0.8379638593094546,0.8250273385679967,0.7780090351726363
2,0.8534679645763867,0.8479164425800438,0.8533415797210568,0.8515892223297838,0.8515125847047436,0.8527921193216450,0.8506449213007781,0.8508327058907891,0.8542952923882257,0.8506001039761930,...,0.8498283496468396,0.8465548922591517,0.8486509985299917,0.8480378975296690,0.8492385536553010,0.8460229106163277,0.8441809185758846,0.8378934961098562,0.8248964719802087,0.7780090351726363
3,0.8535943494317164,0.8486371051593703,0.8497266143200315,0.8515000358538597,0.8547676669893514,0.8535002330500878,0.8518509555053602,0.8495652719515256,0.8477492739593417,0.8524855688214836,...,0.8474588576960309,0.8477322433759994,0.8484538023018178,0.8491430927539350,0.8475260836829084,0.8449096482736366,0.8428673227922986,0.8419768025527948,0.8251008389803163,0.7780090351726363
4,0.8495343479975619,0.8491816356530781,0.8520875909791690,0.8515901186762755,0.8505933813775053,0.8508981391846832,0.8512055860313363,0.8508120899214802,0.8514951059481553,0.8501487935176222,...,0.8468408267900041,0.8479715678892833,0.8501931626689614,0.8487769352120755,0.8457365279122298,0.8440903875802231,0.8436632784769280,0.8395705603958266,0.8259698469040192,0.7780090351726363
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.8545785378796029,0.8480316231042272,0.8520898318453981,0.8509519199741852,0.8520907281918900,0.8505745581011795,0.8505109175002690,0.8504553440177836,0.8514726972858628,0.8494102040084617,...,0.8465867125596070,0.8481387365099853,0.8486016994729484,0.8485515040694130,0.8478501129396580,0.8453874009537126,0.8446048904664587,0.8402011401527375,0.8266497257179736,0.7786965329317701
1996,0.8535522211466066,0.8471921946147505,0.8531103223261984,0.8503128249256033,0.8520315693234376,0.8479092718081102,0.8514762826718297,0.8511410490839338,0.8513714101323007,0.8510039080707039,...,0.8484085368039870,0.8481145351547095,0.8486025958194401,0.8478922412247679,0.8479366103761070,0.8443548097952744,0.8444861245563083,0.8398596321393997,0.8255149510594815,0.7780090351726363
1997,0.8514807644042883,0.8517635617224195,0.8529319493743501,0.8514597002617331,0.8504060449607400,0.8533182747122728,0.8489763723064787,0.8525994048259296,0.8528934064752069,0.8487285325015238,...,0.8485488150299380,0.8471760603778997,0.8506435767810405,0.8499475637302356,0.8465871607328529,0.8414062780108279,0.8449369868416334,0.8395512889462552,0.8258560108995733,0.7781040479007565
1998,0.8560727474812664,0.8487540783765370,0.8532976587429637,0.8517478756588146,0.8523018177906851,0.8480181779068517,0.8486814743107095,0.8499408411315478,0.8510218350005377,0.8511804883295687,...,0.8454752429098991,0.8482498834749560,0.8504199383313613,0.8479814277006921,0.8468309669785953,0.8461726004804417,0.8425325373776487,0.8407021978415976,0.8235994586067191,0.7781040479007565


In [31]:
driver.close()
driver.quit()

In [32]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")
    print()

AUC 신뢰 구간
xgb_0: [0.84952286 0.85637334]

xgb_1: [0.84792219 0.85346202]

xgb_2: [0.84900005 0.85454035]

xgb_3: [0.8480277 0.85356535]

xgb_4: [0.84827272 0.8538117]

xgb_5: [0.8478362 0.85387807]

xgb_6: [0.84739767 0.85356657]

xgb_7: [0.84728759 0.85328258]

xgb_8: [0.84937603 0.85530225]

xgb_9: [0.84776106 0.85309185]

xgb_10: [0.84745507 0.85364788]

xgb_11: [0.84924091 0.85579928]

xgb_12: [0.84831386 0.85509354]

xgb_13: [0.84958835 0.8559393]

xgb_14: [0.85049194 0.85593137]

xgb_15: [0.84804782 0.85456466]

xgb_16: [0.84817145 0.85502266]

xgb_17: [0.84850008 0.85372283]

xgb_18: [0.84829681 0.85558556]

xgb_19: [0.84743286 0.8534112]

xgb_20: [0.84769675 0.85419761]

xgb_21: [0.850318 0.85585861]

xgb_22: [0.84924324 0.85541859]

xgb_23: [0.8473053 0.85242928]

xgb_24: [0.84911154 0.85572808]

xgb_25: [0.84896981 0.85408671]

xgb_26: [0.8478273 0.85482387]

xgb_27: [0.84857389 0.85663787]

xgb_28: [0.84998994 0.85517891]

xgb_29: [0.84838376 0.85392869]

xgb_30: [0.84836989

In [33]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
xgb_0: [0.85 0.86]
xgb_1: [0.85 0.85]
xgb_2: [0.85 0.85]
xgb_3: [0.85 0.85]
xgb_4: [0.85 0.85]
xgb_5: [0.85 0.85]
xgb_6: [0.85 0.85]
xgb_7: [0.85 0.85]
xgb_8: [0.85 0.86]
xgb_9: [0.85 0.85]
xgb_10: [0.85 0.85]
xgb_11: [0.85 0.86]
xgb_12: [0.85 0.86]
xgb_13: [0.85 0.86]
xgb_14: [0.85 0.86]
xgb_15: [0.85 0.85]
xgb_16: [0.85 0.86]
xgb_17: [0.85 0.85]
xgb_18: [0.85 0.86]
xgb_19: [0.85 0.85]
xgb_20: [0.85 0.85]
xgb_21: [0.85 0.86]
xgb_22: [0.85 0.86]
xgb_23: [0.85 0.85]
xgb_24: [0.85 0.86]
xgb_25: [0.85 0.85]
xgb_26: [0.85 0.85]
xgb_27: [0.85 0.86]
xgb_28: [0.85 0.86]
xgb_29: [0.85 0.85]
xgb_30: [0.85 0.85]
xgb_31: [0.85 0.86]
xgb_32: [0.85 0.85]
xgb_33: [0.85 0.85]
xgb_34: [0.85 0.86]
xgb_35: [0.85 0.85]
xgb_36: [0.85 0.85]
xgb_37: [0.85 0.85]
xgb_38: [0.85 0.85]
xgb_39: [0.85 0.85]
xgb_40: [0.85 0.85]
xgb_41: [0.85 0.85]
xgb_42: [0.85 0.85]
xgb_43: [0.85 0.85]
xgb_44: [0.85 0.85]
xgb_45: [0.85 0.85]
xgb_46: [0.85 0.85]
xgb_47: [0.84 0.85]
xgb_48: [0.85 0.85]
xgb_49: [0.85 0.85]


In [34]:
import pingouin as pg
import numpy as np, statsmodels.stats.api as sms

In [35]:
i = 0
while i < 55:  
    base_column = f'xgb_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'xgb_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'xgb_55':
            break
    if compare_column == 'xgb_55' and continue_search:
        break
    i += 1  

xgb_0 and xgb_1:
(0.002311691642161877, 0.0025118553801749058)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0024    5.1e-05     47.246      0.000       0.002       0.003
-------------------------------------------
xgb_0 and xgb_2:
(0.0011845604774493076, 0.0013832086954023118)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0013   5.07e-05     25.343      0.000       0.001       0.001
-------------------------------------------
xgb_0 and xgb_3:
(0.0021120711768254923, 0.0023109071136621635)
                          Test for equality of means                          
          

In [36]:
## xgboost는 X = 14

In [37]:
del new_df

### LGBMClassifier

In [38]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [39]:
df = pd.DataFrame()

In [40]:
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)

In [41]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [42]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [43]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [44]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [45]:
df

,Column_29,Column_44,Column_58,Column_71,Column_84,Column_96,Column_109,Column_122,Column_135,Column_148,...,Column_628,Column_641,Column_654,Column_667,Column_680,Column_693,Column_706,Column_719,Column_732,Column_745
0,"0.8466001577569826, 0.8475942060162777, 0.8512...","0.8491018608153167, 0.8465705783227564, 0.8478...","0.8512562296081173, 0.8532752500806712, 0.8526...","0.8499417374780395, 0.8529458427449714, 0.8522...","0.8534796170807788, 0.8522910616327848, 0.8534...","0.8518052418342834, 0.8533971532035425, 0.8547...","0.849334014556667, 0.851415331110394, 0.850357...","0.8511096769567245, 0.8506785342942168, 0.8535...","0.8500761894517944, 0.8528978882076657, 0.8529...","0.8522543114266251, 0.8491914954644868, 0.8532...",...,"0.8472903445555913, 0.8500407837653724, 0.8502...","0.8502715929869851, 0.8481660750779823, 0.8481...","0.8498556882148363, 0.8502550105768887, 0.8465...","0.8487746943458462, 0.8484210856548708, 0.8481...","0.8477609264637338, 0.8454895844537665, 0.8495...","0.8455285755261555, 0.8461833566383421, 0.8475...","0.846416406726184, 0.845162417984296, 0.844510...","0.8391008748341758, 0.8366865655587823, 0.8382...","0.8263431752178121, 0.8248180416621849, 0.8234...","0.7780090351726363, 0.7781246638700656, 0.7781..."


In [46]:
#pd.set_option('display.max_columns', None)

In [47]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'lgbm_{i}' for i in range(len(df.columns))])

In [48]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [49]:
new_df.shape

(2000, 56)

In [50]:
new_df = new_df.astype(float)

In [51]:
new_df

,lgbm_0,lgbm_1,lgbm_2,lgbm_3,lgbm_4,lgbm_5,lgbm_6,lgbm_7,lgbm_8,lgbm_9,...,lgbm_46,lgbm_47,lgbm_48,lgbm_49,lgbm_50,lgbm_51,lgbm_52,lgbm_53,lgbm_54,lgbm_55
0,0.8466001577569826,0.8491018608153167,0.8512562296081173,0.8499417374780395,0.8534796170807788,0.8518052418342834,0.8493340145566670,0.8511096769567245,0.8500761894517944,0.8522543114266251,...,0.8472903445555913,0.8502715929869851,0.8498556882148363,0.8487746943458462,0.8477609264637338,0.8455285755261555,0.8464164067261840,0.8391008748341758,0.8263431752178121,0.7780090351726363
1,0.8475942060162777,0.8465705783227564,0.8532752500806712,0.8529458427449714,0.8522910616327848,0.8533971532035425,0.8514153311103940,0.8506785342942168,0.8528978882076657,0.8491914954644868,...,0.8500407837653724,0.8481660750779823,0.8502550105768887,0.8484210856548708,0.8454895844537665,0.8461833566383421,0.8451624179842960,0.8366865655587823,0.8248180416621849,0.7781246638700656
2,0.8512737083647053,0.8478900003585386,0.8526863504356244,0.8522059087160733,0.8534975440106127,0.8547461546735506,0.8503576422501883,0.8535952457782079,0.8529337420673335,0.8532349144885446,...,0.8502953461690150,0.8481835538345702,0.8465880570793446,0.8481602488257861,0.8495168692409738,0.8475798644724107,0.8445103259115843,0.8382905776056792,0.8234901043347317,0.7781040479007565
3,0.8513866480226597,0.8502079523860744,0.8500255458750134,0.8519813739199025,0.8583046502455989,0.8552059804237926,0.8535486357606397,0.8521835000537809,0.8483130759026209,0.8540245957477323,...,0.8507350041231939,0.8503692947545805,0.8477340360689828,0.8496571474669248,0.8478783478541465,0.8451848266465884,0.8443794593237962,0.8415331110394033,0.8257650317306658,0.7781040479007565
4,0.8474740955863898,0.8485201319422035,0.8523520131942204,0.8523681474310710,0.8519576207378725,0.8519289376501380,0.8530421999928293,0.8518554372378188,0.8513552758954502,0.8487827614642716,...,0.8492658922232978,0.8490946900433832,0.8492174895127462,0.8485286472338747,0.8463312538094725,0.8457141192499373,0.8429107955971460,0.8395114015273745,0.8264740418056004,0.7781040479007565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.8524425441898822,0.8481499408411316,0.8525263525868561,0.8525312824925604,0.8563730235559859,0.8504069413072317,0.8509626761320856,0.8512665375927719,0.8516689971675451,0.8500735004123194,...,0.8508192606934137,0.8487419776988994,0.8454595568462946,0.8489714424007744,0.8485573303216091,0.8469461475027787,0.8436794127137786,0.8396095514682156,0.8272834426876053,0.7786965329317701
1996,0.8506579183249077,0.8472679358932989,0.8517680434548779,0.8497418522103904,0.8504365207414577,0.8507887849126958,0.8534401778351439,0.8500080671184252,0.8500609515614357,0.8526594600408735,...,0.8503710874475637,0.8511804883295687,0.8482198558674842,0.8482292675056470,0.8477237280843283,0.8456554085547310,0.8442615897601377,0.8407488078591659,0.8259877738338532,0.7763561722419419
1997,0.8489181097845182,0.8498521028288695,0.8521324083037538,0.8533326162561399,0.8493994478505611,0.8536839840808864,0.8511392563909504,0.8539466136029544,0.8511553906278011,0.8512002079523862,...,0.8501743393926356,0.8485618120540677,0.8501505862106056,0.8490055035674590,0.8463491807393067,0.8455653257323150,0.8450611308307339,0.8399671937184038,0.8295731598006525,0.7786965329317701
1998,0.8549585887920835,0.8484977232799109,0.8522771682621635,0.8526612527338567,0.8549039116560898,0.8506901867986089,0.8514359470797030,0.8509402674697931,0.8522032196765981,0.8536983256247535,...,0.8487339105804740,0.8501958517084365,0.8497355777849487,0.8486716144993008,0.8475942060162777,0.8455205084077301,0.8446900433831701,0.8406466243591123,0.8236397941988455,0.7781040479007565


In [75]:
driver.close()
driver.quit()

In [53]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
lgbm_0: [0.84775104 0.85516119]
lgbm_1: [0.84759123 0.85526135]
lgbm_2: [0.84884984 0.85409821]
lgbm_3: [0.84896299 0.85408472]
lgbm_4: [0.84836536 0.85531905]
lgbm_5: [0.84889563 0.85535914]
lgbm_6: [0.8484923 0.85580089]
lgbm_7: [0.84877823 0.85567499]
lgbm_8: [0.84895605 0.85464628]
lgbm_9: [0.84743132 0.85487993]
lgbm_10: [0.84898406 0.85453372]
lgbm_11: [0.846093 0.85387502]
lgbm_12: [0.85008549 0.85614304]
lgbm_13: [0.84938075 0.85429991]
lgbm_14: [0.84949327 0.85670494]
lgbm_15: [0.84860524 0.85566507]
lgbm_16: [0.84851088 0.8553076]
lgbm_17: [0.84861398 0.85576203]
lgbm_18: [0.84371524 0.85453143]
lgbm_19: [0.84842736 0.85550339]
lgbm_20: [0.85002059 0.85593751]
lgbm_21: [0.8483552 0.85527251]
lgbm_22: [0.84841519 0.85526276]
lgbm_23: [0.84693427 0.85481251]
lgbm_24: [0.84822998 0.85461825]
lgbm_25: [0.84903004 0.85590184]
lgbm_26: [0.84811852 0.85498402]
lgbm_27: [0.84701723 0.85519292]
lgbm_28: [0.84893198 0.85536401]
lgbm_29: [0.84654573 0.8541357]
lgbm_30: [0.8493

In [54]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
lgbm_0: [0.85 0.86]
lgbm_1: [0.85 0.86]
lgbm_2: [0.85 0.85]
lgbm_3: [0.85 0.85]
lgbm_4: [0.85 0.86]
lgbm_5: [0.85 0.86]
lgbm_6: [0.85 0.86]
lgbm_7: [0.85 0.86]
lgbm_8: [0.85 0.85]
lgbm_9: [0.85 0.85]
lgbm_10: [0.85 0.85]
lgbm_11: [0.85 0.85]
lgbm_12: [0.85 0.86]
lgbm_13: [0.85 0.85]
lgbm_14: [0.85 0.86]
lgbm_15: [0.85 0.86]
lgbm_16: [0.85 0.86]
lgbm_17: [0.85 0.86]
lgbm_18: [0.84 0.85]
lgbm_19: [0.85 0.86]
lgbm_20: [0.85 0.86]
lgbm_21: [0.85 0.86]
lgbm_22: [0.85 0.86]
lgbm_23: [0.85 0.85]
lgbm_24: [0.85 0.85]
lgbm_25: [0.85 0.86]
lgbm_26: [0.85 0.85]
lgbm_27: [0.85 0.86]
lgbm_28: [0.85 0.86]
lgbm_29: [0.85 0.85]
lgbm_30: [0.85 0.86]
lgbm_31: [0.85 0.86]
lgbm_32: [0.84 0.85]
lgbm_33: [0.85 0.86]
lgbm_34: [0.85 0.85]
lgbm_35: [0.85 0.86]
lgbm_36: [0.85 0.86]
lgbm_37: [0.85 0.86]
lgbm_38: [0.85 0.85]
lgbm_39: [0.85 0.86]
lgbm_40: [0.85 0.85]
lgbm_41: [0.85 0.85]
lgbm_42: [0.85 0.85]
lgbm_43: [0.85 0.85]
lgbm_44: [0.85 0.85]
lgbm_45: [0.85 0.85]
lgbm_46: [0.85 0.85]
lgbm_47: [0.8

In [55]:
i = 0
while i < 55:  
    base_column = f'lgbm_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'lgbm_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'lgbm_55':
            break
    if compare_column == 'lgbm_55' and continue_search:
        break
    i += 1  

lgbm_0 and lgbm_1:
(-2.1871061610230394e-05, 0.00022074435407049692)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1   9.944e-05   6.19e-05      1.607      0.108   -2.19e-05       0.000
-------------------------------------------
lgbm_1 and lgbm_2:
(-0.00024715935581124684, -3.819299799497623e-05)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0001   5.33e-05     -2.677      0.007      -0.000   -3.82e-05
-------------------------------------------
lgbm_2 and lgbm_3:
(-8.285200661845468e-05, 8.213582577182752e-05)
                          Test for equality of means                       

In [56]:
### lgbm = 37

In [57]:
del new_df

### GradientBoosting

In [58]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [59]:
df = pd.DataFrame()

In [60]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

In [61]:
df = df.dropna(axis=1, how='all')

In [62]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [63]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [64]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [65]:
df

,Column_25,Column_38,Column_50,Column_62,Column_74,Column_85,Column_97,Column_109,Column_121,Column_133,...,Column_585,Column_598,Column_611,Column_624,Column_637,Column_650,Column_663,Column_676,Column_689,Column_702
0,"0.8505673873292461, 0.846640493349109, 0.85137...","0.8519217668782044, 0.8476766698935139, 0.8495...","0.8519612061238392, 0.8533675737693163, 0.8532...","0.8515569538560827, 0.8536418557957764, 0.8517...","0.8508999318776667, 0.8532098167867771, 0.8537...","0.8519880965185903, 0.8545462694059016, 0.8519...","0.8494953569251731, 0.8482108924025672, 0.8488...","0.8533693664622997, 0.8533819153131834, 0.8527...","0.8506588146713993, 0.8524299953389981, 0.8535...","0.8531291456025242, 0.8524210318740812, 0.8530...",...,"0.8456769208705317, 0.8460614535154711, 0.8486...","0.8499726614320031, 0.8461143379584812, 0.8483...","0.8479679825033164, 0.8491318884227887, 0.8465...","0.8483480334157973, 0.8478900003585386, 0.8476...","0.8460457674518662, 0.843726470904593, 0.84802...","0.8419476712918145, 0.8447079703130043, 0.8456...","0.8433491090315872, 0.8407653902692626, 0.8404...","0.8390098956652684, 0.8346863683625543, 0.8394...","0.8245827507081138, 0.8261997597791402, 0.8280...","0.777830662220788, 0.7777437166110932, 0.77755..."


In [66]:
df.shape

(1, 56)

In [67]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'gb_{i}' for i in range(len(df.columns))])

In [71]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [68]:
new_df.shape

(2000, 56)

In [69]:
new_df = new_df.astype(float)

In [70]:
new_df

,gb_0,gb_1,gb_2,gb_3,gb_4,gb_5,gb_6,gb_7,gb_8,gb_9,...,gb_46,gb_47,gb_48,gb_49,gb_50,gb_51,gb_52,gb_53,gb_54,gb_55
0,0.8505673873292461,0.8519217668782044,0.8519612061238392,0.8515569538560827,0.8508999318776667,0.8519880965185903,0.8494953569251731,0.8533693664622997,0.8506588146713993,0.8531291456025242,...,0.8456769208705317,0.8499726614320031,0.8479679825033164,0.8483480334157973,0.8460457674518662,0.8419476712918145,0.8433491090315872,0.8390098956652684,0.8245827507081138,0.7778306622207880
1,0.8466404933491090,0.8476766698935139,0.8533675737693163,0.8536418557957764,0.8532098167867771,0.8545462694059016,0.8482108924025672,0.8533819153131834,0.8524299953389981,0.8524210318740812,...,0.8460614535154711,0.8461143379584812,0.8491318884227887,0.8478900003585386,0.8437264709045930,0.8447079703130043,0.8407653902692626,0.8346863683625543,0.8261997597791402,0.7777437166110932
2,0.8513749955182675,0.8495715463769675,0.8532268473701193,0.8517846258649744,0.8537754114230397,0.8519782367071816,0.8488455057186906,0.8527948083611201,0.8535871786597827,0.8530717794270554,...,0.8486124556308485,0.8483959879531032,0.8465629593775770,0.8476188555447993,0.8480266931985228,0.8456612348069269,0.8404489799576924,0.8394670323760351,0.8280525079774838,0.7775545875013445
3,0.8478442866874620,0.8507977483776128,0.8503146176185867,0.8496620773726291,0.8575400666881789,0.8549316983973325,0.8520526334659928,0.8540407299845829,0.8474481015381305,0.8516672044745618,...,0.8475852425513608,0.8479348176831236,0.8458543974758883,0.8494438170019003,0.8467050302965115,0.8437264709045927,0.8409002904162632,0.8403261804883295,0.8272955433652432,0.7786364777168262
4,0.8475458033057258,0.8490050553942132,0.8529220895629415,0.8508533218600981,0.8515461976981823,0.8526496002294648,0.8533604029973827,0.8496432540963035,0.8499498045964649,0.8494935642321897,...,0.8484363235452297,0.8493246029185042,0.8484134667096914,0.8460063282062315,0.8448486967122011,0.8437860779462909,0.8408469578000071,0.8402880857624322,0.8294754580330571,0.7779256749489082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.8524048976372306,0.8483103868631456,0.8525276971065935,0.8510926463733821,0.8548331002832456,0.8507520347065362,0.8494281309382955,0.8494657774909469,0.8496988275787889,0.8491386110214765,...,0.8494711555698971,0.8472334265533685,0.8415012907389481,0.8449827005127102,0.8455146821555339,0.8451722777957047,0.8409451077408483,0.8385074934566704,0.8292173102434478,0.7782904879710301
1996,0.8520884873256607,0.8497364741314402,0.8499507009429567,0.8502733856799684,0.8526316732996307,0.8508434620486895,0.8510128715356207,0.8500358538596680,0.8477680972356675,0.8511831773690438,...,0.8471912982682586,0.8484842780825356,0.8478519056326415,0.8428216091212219,0.8439846186942025,0.8444027643325803,0.8397269728586282,0.8403575526155391,0.8269580689111183,0.7762961170269980
1997,0.8484555949948013,0.8500089634649171,0.8547506364060090,0.8534903732386792,0.8526997956329998,0.8526487038829729,0.8501936108422071,0.8549738266824424,0.8520687677028432,0.8475897242838191,...,0.8473333691871929,0.8449383313613712,0.8503078950198989,0.8471469291169195,0.8448832060521315,0.8437860779462909,0.8432939837223478,0.8386518052418344,0.8290954071205765,0.7786364777168262
1998,0.8543419024057941,0.8483749238105482,0.8542038650460722,0.8492578251048726,0.8540685167258255,0.8521010361765444,0.8499031945788964,0.8519737549747229,0.8495249363593991,0.8544171955110968,...,0.8446295399949806,0.8498588254275572,0.8493837617869563,0.8445304937076477,0.8473033415797211,0.8444014198128428,0.8402710551790900,0.8419633573554192,0.8253374744541249,0.7784975440106127


In [70]:
driver.close()
driver.quit()

In [71]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
gb_0: [0.85 0.86]
gb_1: [0.85 0.85]
gb_2: [0.85 0.85]
gb_3: [0.85 0.86]
gb_4: [0.85 0.86]
gb_5: [0.85 0.86]
gb_6: [0.85 0.86]
gb_7: [0.85 0.86]
gb_8: [0.85 0.86]
gb_9: [0.85 0.86]
gb_10: [0.85 0.86]
gb_11: [0.85 0.86]
gb_12: [0.85 0.86]
gb_13: [0.85 0.86]
gb_14: [0.85 0.86]
gb_15: [0.85 0.86]
gb_16: [0.85 0.85]
gb_17: [0.85 0.86]
gb_18: [0.85 0.86]
gb_19: [0.85 0.86]
gb_20: [0.85 0.85]
gb_21: [0.85 0.86]
gb_22: [0.85 0.85]
gb_23: [0.85 0.85]
gb_24: [0.85 0.86]
gb_25: [0.85 0.85]
gb_26: [0.85 0.85]
gb_27: [0.85 0.85]
gb_28: [0.85 0.85]
gb_29: [0.85 0.85]
gb_30: [0.85 0.85]
gb_31: [0.85 0.85]
gb_32: [0.85 0.85]
gb_33: [0.85 0.85]
gb_34: [0.85 0.85]
gb_35: [0.85 0.85]
gb_36: [0.85 0.85]
gb_37: [0.85 0.85]
gb_38: [0.85 0.85]
gb_39: [0.85 0.85]
gb_40: [0.85 0.85]
gb_41: [0.85 0.85]
gb_42: [0.85 0.85]
gb_43: [0.85 0.85]
gb_44: [0.85 0.85]
gb_45: [0.84 0.85]
gb_46: [0.84 0.85]
gb_47: [0.84 0.85]
gb_48: [0.84 0.85]
gb_49: [0.84 0.85]
gb_50: [0.84 0.85]
gb_51: [0.84 0.85]
gb_52: [0.84

In [72]:
i = 0
while i < 55:  
    base_column = f'gb_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'gb_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'gb_55':
            break
    if compare_column == 'gb_55' and continue_search:
        break
    i += 1  

gb_0 and gb_1:
(-8.006367673681137e-05, 0.00014987517506966154)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1   3.491e-05   5.86e-05      0.595      0.552   -8.01e-05       0.000
-------------------------------------------
gb_1 and gb_2:
(-0.0001532955213528072, 7.731536646395216e-05)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1  -3.799e-05   5.88e-05     -0.646      0.518      -0.000    7.73e-05
-------------------------------------------
gb_2 and gb_3:
(-0.00013324603447664637, 9.575320524848791e-05)
                          Test for equality of means                          
          

In [73]:
## gb = 5

In [74]:
del new_df

### ExtraTrees

In [76]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [77]:
df = pd.DataFrame()

In [78]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

In [79]:
df = df.dropna(axis=1, how='all')

In [80]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [81]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [82]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [83]:
#pd.set_option('display.max_columns', None)

In [84]:
df

,Column_22,Column_34,Column_46,Column_58,Column_69,Column_81,Column_93,Column_105,Column_117,Column_129,...,Column_595,Column_608,Column_621,Column_634,Column_647,Column_660,Column_673,Column_686,Column_699,Column_712
0,"0.8528432110716718, 0.8489028718941595, 0.8547...","0.8486958158545767, 0.8472240149152056, 0.8468...","0.8498440357104442, 0.8558970635688933, 0.8539...","0.8490677996486321, 0.852723100641784, 0.85107...","0.8512477143164462, 0.8514682155534044, 0.8488...","0.8457136710766914, 0.8492658922232978, 0.8476...","0.8507063210354594, 0.8521646767774551, 0.8514...","0.8536786060019362, 0.8521279265712953, 0.8505...","0.8482377827973182, 0.8496047111971604, 0.8542...","0.8539259976336453, 0.8489207988239935, 0.8504...",...,"0.8505575275178372, 0.8493479079272884, 0.8493...","0.8503881180309061, 0.8472007099064213, 0.8456...","0.8473100641784089, 0.8469954465598221, 0.8477...","0.8488593990893121, 0.849166845935965, 0.84511...","0.8477748198343551, 0.8438506148936933, 0.8459...","0.8474879889570113, 0.8459767487720053, 0.8471...","0.8426020042307554, 0.8399792943960417, 0.8421...","0.8420641963357356, 0.8413775949230935, 0.8401...","0.8340696819762647, 0.834041895235022, 0.83202...","0.7781246638700656, 0.7781246638700656, 0.7781..."


In [85]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'ext_{i}' for i in range(len(df.columns))])

In [86]:
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data)

112000

In [87]:
new_df.shape

(2000, 56)

In [88]:
new_df = new_df.astype(float)

In [89]:
new_df

,ext_0,ext_1,ext_2,ext_3,ext_4,ext_5,ext_6,ext_7,ext_8,ext_9,...,ext_46,ext_47,ext_48,ext_49,ext_50,ext_51,ext_52,ext_53,ext_54,ext_55
0,0.8528432110716718,0.8486958158545767,0.8498440357104442,0.8490677996486321,0.8512477143164462,0.8457136710766914,0.8507063210354594,0.8536786060019362,0.8482377827973182,0.8539259976336453,...,0.8505575275178372,0.8503881180309061,0.8473100641784089,0.8488593990893121,0.8477748198343551,0.8474879889570113,0.8426020042307554,0.8420641963357356,0.8340696819762647,0.7781246638700656
1,0.8489028718941595,0.8472240149152056,0.8558970635688933,0.8527231006417840,0.8514682155534044,0.8492658922232978,0.8521646767774551,0.8521279265712953,0.8496047111971604,0.8489207988239935,...,0.8493479079272884,0.8472007099064213,0.8469954465598221,0.8491668459359650,0.8438506148936933,0.8459767487720053,0.8399792943960417,0.8413775949230935,0.8340418952350220,0.7781246638700656
2,0.8547936610376107,0.8468125918755154,0.8539000035853859,0.8510702377110896,0.8488060664730559,0.8476712918145639,0.8514296726542612,0.8505333261625615,0.8542845362303252,0.8504248682370656,...,0.8493039869491950,0.8456652683661396,0.8477600301172421,0.8451180488329568,0.8459821268509555,0.8471384138252482,0.8421932702305404,0.8401836613961493,0.8320242192822057,0.7781246638700656
3,0.8542155175504642,0.8481069162095300,0.8512199275752035,0.8494128930479365,0.8523860743609050,0.8526729052382490,0.8533263418306980,0.8521243411853286,0.8500430246316016,0.8501093542719873,...,0.8497216844143272,0.8506059302283892,0.8480764404288121,0.8472208777024848,0.8463128787063927,0.8441199670144490,0.8430654153669642,0.8422820085332186,0.8313017640098956,0.7787171489010791
4,0.8517505646982897,0.8476488831522713,0.8517451866193396,0.8491843246925532,0.8503083431931446,0.8482386791438099,0.8546072209673372,0.8530646086551218,0.8512432325839877,0.8501514825570973,...,0.8482194076942382,0.8476426087268294,0.8488661216879997,0.8504091821734611,0.8461694632677208,0.8426835717615002,0.8406318346419992,0.8431837331038687,0.8324437094403213,0.7781246638700656
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.8511365673514754,0.8463438026603564,0.8514153311103940,0.8510971281058406,0.8500026890394752,0.8494998386576315,0.8521843964002723,0.8511410490839340,0.8496172600480441,0.8511320856190170,...,0.8489100426660930,0.8503164103115701,0.8462864364848877,0.8463236348642931,0.8447335161880177,0.8471733713384246,0.8398753182030045,0.8410692517299487,0.8320215302427306,0.7786965329317701
1996,0.8527723996988276,0.8469398730773368,0.8489055609336345,0.8525259044136102,0.8524792943960418,0.8509590907461189,0.8512423362374961,0.8512557814348714,0.8479832203936755,0.8493510451400094,...,0.8502290165286294,0.8491305439030511,0.8490570434907317,0.8473329210139471,0.8444242766483813,0.8469277723996987,0.8408012441289305,0.8437412606217058,0.8343726470904593,0.7763767882112509
1997,0.8536705388835109,0.8464468825069019,0.8528647233874727,0.8521413717686708,0.8492757520347065,0.8472087770248466,0.8501093542719874,0.8534097020544261,0.8514691118998960,0.8521243411853286,...,0.8492089742210749,0.8479599153848911,0.8471980208669464,0.8465423434082681,0.8464446416406727,0.8440110609157077,0.8414259976336453,0.8428843533756408,0.8321129575848840,0.7786965329317701
1998,0.8503441970528128,0.8489100426660929,0.8526720088917573,0.8506211681187481,0.8497293033595067,0.8486590656484170,0.8500824638772364,0.8507009429565092,0.8521790183213223,0.8495840952278513,...,0.8513400380050913,0.8523188483740274,0.8476345416084042,0.8499117098705675,0.8469663152988420,0.8466902405793983,0.8421977519629987,0.8409356961026855,0.8337953999498046,0.7781040479007565


In [90]:
driver.close()
driver.quit()

In [91]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
ext_0: [0.84795976 0.85454428]
ext_1: [0.84567683 0.85234578]
ext_2: [0.84742513 0.85479917]
ext_3: [0.84630954 0.85386693]
ext_4: [0.84654228 0.85388017]
ext_5: [0.8456581 0.85247938]
ext_6: [0.84780931 0.85451586]
ext_7: [0.84781356 0.85489446]
ext_8: [0.84816646 0.85479817]
ext_9: [0.84673367 0.85399681]
ext_10: [0.84683473 0.85409933]
ext_11: [0.84754623 0.85468738]
ext_12: [0.84743714 0.85473903]
ext_13: [0.84703771 0.85406063]
ext_14: [0.84777233 0.85485038]
ext_15: [0.84678017 0.85416357]
ext_16: [0.84628444 0.85286681]
ext_17: [0.8464854 0.85338561]
ext_18: [0.84761751 0.85454035]
ext_19: [0.84696481 0.85395204]
ext_20: [0.84741568 0.85470956]
ext_21: [0.84777863 0.85480132]
ext_22: [0.84624034 0.85311232]
ext_23: [0.84749547 0.85418623]
ext_24: [0.84827682 0.85525037]
ext_25: [0.84927586 0.8558559]
ext_26: [0.84843231 0.85543939]
ext_27: [0.84874597 0.85578424]
ext_28: [0.84803035 0.85550468]
ext_29: [0.84840491 0.85510667]
ext_30: [0.84801121 0.85492854]
ext_31: [0.

In [92]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
ext_0: [0.85 0.85]
ext_1: [0.85 0.85]
ext_2: [0.85 0.85]
ext_3: [0.85 0.85]
ext_4: [0.85 0.85]
ext_5: [0.85 0.85]
ext_6: [0.85 0.85]
ext_7: [0.85 0.85]
ext_8: [0.85 0.85]
ext_9: [0.85 0.85]
ext_10: [0.85 0.85]
ext_11: [0.85 0.85]
ext_12: [0.85 0.85]
ext_13: [0.85 0.85]
ext_14: [0.85 0.85]
ext_15: [0.85 0.85]
ext_16: [0.85 0.85]
ext_17: [0.85 0.85]
ext_18: [0.85 0.85]
ext_19: [0.85 0.85]
ext_20: [0.85 0.85]
ext_21: [0.85 0.85]
ext_22: [0.85 0.85]
ext_23: [0.85 0.85]
ext_24: [0.85 0.86]
ext_25: [0.85 0.86]
ext_26: [0.85 0.86]
ext_27: [0.85 0.86]
ext_28: [0.85 0.86]
ext_29: [0.85 0.86]
ext_30: [0.85 0.85]
ext_31: [0.85 0.85]
ext_32: [0.85 0.85]
ext_33: [0.85 0.85]
ext_34: [0.85 0.86]
ext_35: [0.85 0.86]
ext_36: [0.85 0.86]
ext_37: [0.85 0.86]
ext_38: [0.85 0.85]
ext_39: [0.85 0.86]
ext_40: [0.85 0.85]
ext_41: [0.85 0.85]
ext_42: [0.85 0.85]
ext_43: [0.85 0.85]
ext_44: [0.85 0.85]
ext_45: [0.84 0.85]
ext_46: [0.85 0.85]
ext_47: [0.85 0.85]
ext_48: [0.84 0.85]
ext_49: [0.84 0.85]


In [93]:
i = 0
while i < 55:  
    base_column = f'ext_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'ext_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'ext_55':
            break
    if compare_column == 'ext_55' and continue_search:
        break
    i += 1  

ext_0 and ext_1:
(0.0021864690783600163, 0.0023992234389395503)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0023   5.43e-05     42.258      0.000       0.002       0.002
-------------------------------------------
ext_0 and ext_2:
(-6.138874949940498e-05, 0.00016275298885976252)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1   5.068e-05   5.72e-05      0.887      0.375   -6.14e-05       0.000
-------------------------------------------
ext_1 and ext_2:
(-0.0023525813594259736, -0.0021317469185132354)
                          Test for equality of means                          
     

In [94]:
### ext = 25

In [95]:
del new_df

### rf

In [115]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [97]:
df = pd.DataFrame()

In [98]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [99]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [100]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [101]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [102]:
df

,Column_23,Column_35,Column_47,Column_59,Column_71,Column_83,Column_95,Column_107,Column_120,Column_132,...,Column_587,Column_600,Column_613,Column_626,Column_639,Column_652,Column_665,Column_678,Column_691,Column_704
0,"0.8521153777204117, 0.8480459646480945, 0.8499...","0.8494075149689865, 0.8464092359542504, 0.8448...","0.8486016994729483, 0.8524013122512638, 0.8495...","0.8495563084866086, 0.8512808791366391, 0.8460...","0.8505100211537773, 0.8472347710731061, 0.8488...","0.8484896561614859, 0.8496423577498117, 0.8497...","0.8488723961134417, 0.8510693413645979, 0.8509...","0.8527150335233588, 0.8526334659926142, 0.8482...","0.8482512279946938, 0.8484771073106021, 0.8503...","0.8517783514395325, 0.8482449535692517, 0.8481...",...,"0.8462317593488939, 0.844466853106737, 0.84604...","0.8497095837366891, 0.8479491592269908, 0.8447...","0.847321716682801, 0.8479939765515758, 0.84601...","0.8455819081424116, 0.8454223584668887, 0.8440...","0.8461560180703451, 0.8437305044638056, 0.8473...","0.8410571510523108, 0.8417011760065971, 0.8410...","0.8444950880212254, 0.8398089885626188, 0.8412...","0.8426513032877989, 0.8421977519629987, 0.8423...","0.8340051450288624, 0.8347329783801226, 0.8316...","0.7780090351726363, 0.7787171489010791, 0.7786..."


In [103]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'rf_{i}' for i in range(len(df.columns))])

In [104]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [105]:
new_df.shape

(2000, 56)

In [106]:
new_df = new_df.astype(float)

In [107]:
new_df

,rf_0,rf_1,rf_2,rf_3,rf_4,rf_5,rf_6,rf_7,rf_8,rf_9,...,rf_46,rf_47,rf_48,rf_49,rf_50,rf_51,rf_52,rf_53,rf_54,rf_55
0,0.8521153777204117,0.8494075149689865,0.8486016994729483,0.8495563084866086,0.8505100211537773,0.8484896561614859,0.8488723961134417,0.8527150335233588,0.8482512279946938,0.8517783514395325,...,0.8462317593488939,0.8497095837366891,0.8473217166828010,0.8455819081424116,0.8461560180703451,0.8410571510523108,0.8444950880212254,0.8426513032877989,0.8340051450288624,0.7780090351726363
1,0.8480459646480945,0.8464092359542504,0.8524013122512638,0.8512808791366391,0.8472347710731061,0.8496423577498117,0.8510693413645979,0.8526334659926142,0.8484771073106021,0.8482449535692517,...,0.8444668531067370,0.8479491592269908,0.8479939765515758,0.8454223584668887,0.8437305044638056,0.8417011760065971,0.8398089885626188,0.8421977519629987,0.8347329783801226,0.7787171489010791
2,0.8499112616973217,0.8448253917034170,0.8495751317629342,0.8460793804453048,0.8488974938152092,0.8497705352981249,0.8509994263382452,0.8482987343587536,0.8503343372414041,0.8481364956437560,...,0.8460403893729160,0.8447160374314295,0.8460152916711484,0.8440496038148506,0.8473463662113226,0.8410486357606396,0.8412825821949734,0.8423707468358969,0.8316298268258578,0.7786965329317701
3,0.8520131942203578,0.8485022050123696,0.8455254383134344,0.8464459861604102,0.8518419920404431,0.8499390484385643,0.8525097701767594,0.8507367968161773,0.8476963895163313,0.8509743286364777,...,0.8475018823276326,0.8485492632031838,0.8472316338603850,0.8455240937936968,0.8446550858699938,0.8404678032340180,0.8422443619805672,0.8426979133053674,0.8316782295364096,0.7786965329317701
4,0.8482996307052455,0.8461098562260227,0.8485210282886951,0.8463446990068482,0.8458893549890646,0.8504607220967337,0.8509411638162847,0.8507744433688287,0.8462873328313794,0.8492838191531317,...,0.8462227958839769,0.8474835072245527,0.8483892653544155,0.8459225198092575,0.8439971675450862,0.8403414183786885,0.8409379369689147,0.8442261840737155,0.8331948478003657,0.7786965329317701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.8506839123731669,0.8462873328313792,0.8502169158509914,0.8498323832060521,0.8503522641712381,0.8492273493241548,0.8508667670574739,0.8493555268724678,0.8481804166218493,0.8487011939335269,...,0.8454228066401348,0.8503383708006167,0.8456495823025348,0.8455854935283784,0.8440881467139937,0.8386370155247213,0.8401124018500591,0.8433567279767666,0.8322330680147718,0.7786965329317701
1996,0.8492425872145136,0.8453282420852605,0.8491699831486859,0.8511939335269442,0.8500851529167115,0.8471164533362017,0.8497687426051415,0.8492587214513643,0.8474176257574126,0.8473091678319171,...,0.8459364131798788,0.8490010218350005,0.8486097665913735,0.8445909970958374,0.8437654619769819,0.8407488078591659,0.8406461761858665,0.8426584740597325,0.8348593632354523,0.7763561722419419
1997,0.8517317414219641,0.8489620307626116,0.8508927611057331,0.8475377361873007,0.8471809902836041,0.8497418522103903,0.8493465634075509,0.8529122297515328,0.8527741923918110,0.8502213975834498,...,0.8482050661503711,0.8477900577247142,0.8473521924635188,0.8470200960883438,0.8438721272094940,0.8363325086945610,0.8395741457817935,0.8431111290380409,0.8344237388404862,0.7786965329317701
1998,0.8486025958194400,0.8472177404897637,0.8488383349467570,0.8485676383062636,0.8516035638736509,0.8476345416084041,0.8487890358897136,0.8491439891004267,0.8501192140833961,0.8495401742497579,...,0.8451359757627909,0.8511432899501631,0.8491982180631744,0.8447908823634864,0.8446855616507117,0.8428171273887635,0.8402656771001398,0.8421816177261482,0.8343583055465921,0.7786965329317701


In [108]:
driver.close()
driver.quit()

In [109]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
rf_0: [0.84516051 0.8524334]
rf_1: [0.84534162 0.85321748]
rf_2: [0.84552537 0.85243197]
rf_3: [0.84430365 0.85247932]
rf_4: [0.84446735 0.85267851]
rf_5: [0.84537382 0.85218637]
rf_6: [0.84684921 0.85376945]
rf_7: [0.84665481 0.85375177]
rf_8: [0.84612478 0.85309627]
rf_9: [0.8458468 0.852553]
rf_10: [0.84522785 0.85323863]
rf_11: [0.84633567 0.85404714]
rf_12: [0.84627989 0.85408333]
rf_13: [0.84637067 0.85424158]
rf_14: [0.84551419 0.85336076]
rf_15: [0.84619792 0.85392981]
rf_16: [0.84656455 0.85406412]
rf_17: [0.84632119 0.85379946]
rf_18: [0.8462476 0.85380219]
rf_19: [0.84652737 0.85409478]
rf_20: [0.84755822 0.85414567]
rf_21: [0.84750724 0.85498225]
rf_22: [0.84679023 0.8530604]
rf_23: [0.84752586 0.85491469]
rf_24: [0.84729178 0.85443185]
rf_25: [0.84809166 0.85419775]
rf_26: [0.8482895 0.85546166]
rf_27: [0.84906146 0.85582847]
rf_28: [0.8487608 0.85604238]
rf_29: [0.8487204 0.85581122]
rf_30: [0.84817856 0.8555063]
rf_31: [0.84909182 0.85556474]
rf_32: [0.84763167

In [110]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
rf_0: [0.85 0.85]
rf_1: [0.85 0.85]
rf_2: [0.85 0.85]
rf_3: [0.84 0.85]
rf_4: [0.84 0.85]
rf_5: [0.85 0.85]
rf_6: [0.85 0.85]
rf_7: [0.85 0.85]
rf_8: [0.85 0.85]
rf_9: [0.85 0.85]
rf_10: [0.85 0.85]
rf_11: [0.85 0.85]
rf_12: [0.85 0.85]
rf_13: [0.85 0.85]
rf_14: [0.85 0.85]
rf_15: [0.85 0.85]
rf_16: [0.85 0.85]
rf_17: [0.85 0.85]
rf_18: [0.85 0.85]
rf_19: [0.85 0.85]
rf_20: [0.85 0.85]
rf_21: [0.85 0.85]
rf_22: [0.85 0.85]
rf_23: [0.85 0.85]
rf_24: [0.85 0.85]
rf_25: [0.85 0.85]
rf_26: [0.85 0.86]
rf_27: [0.85 0.86]
rf_28: [0.85 0.86]
rf_29: [0.85 0.86]
rf_30: [0.85 0.86]
rf_31: [0.85 0.86]
rf_32: [0.85 0.85]
rf_33: [0.85 0.85]
rf_34: [0.85 0.85]
rf_35: [0.85 0.86]
rf_36: [0.85 0.85]
rf_37: [0.85 0.85]
rf_38: [0.85 0.85]
rf_39: [0.85 0.85]
rf_40: [0.85 0.85]
rf_41: [0.85 0.85]
rf_42: [0.85 0.85]
rf_43: [0.85 0.85]
rf_44: [0.85 0.85]
rf_45: [0.85 0.85]
rf_46: [0.84 0.85]
rf_47: [0.85 0.85]
rf_48: [0.84 0.85]
rf_49: [0.84 0.85]
rf_50: [0.84 0.85]
rf_51: [0.84 0.84]
rf_52: [0.84

In [111]:
i = 0
while i < 55:  
    base_column = f'rf_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'rf_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'rf_55':
            break
    if compare_column == 'rf_55' and continue_search:
        break
    i += 1   

rf_0 and rf_1:
(-0.0006859316346911799, -0.0004414284456213762)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1     -0.0006   6.24e-05     -9.040      0.000      -0.001      -0.000
-------------------------------------------
rf_1 and rf_2:
(0.0002202989501051164, 0.00045989358512132624)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0003   6.11e-05      5.566      0.000       0.000       0.000
-------------------------------------------
rf_1 and rf_3:
(0.0007570870647426275, 0.0010157948684972507)
                          Test for equality of means                          
            

In [112]:
## rf = 35

In [113]:
del new_df

### dt

In [116]:
driver = webdriver.Chrome(options=options)
driver.get(url='.html')

In [117]:
df = pd.DataFrame()

In [118]:
for i in range(0, 775):
    xpath = f'/html/body/div[{i}]/div[2]/div[2]/div/div[2]/pre'
    try:
        element = driver.find_element("xpath", xpath)
        text = element.text.strip('[]')
        df[f'Column_{i}'] = [text]
    except:
        df[f'Column_{i}'] = [None]

df = df.dropna(axis=1, how='all')

In [119]:
cols_to_drop = [col for col in df.columns if df[col].apply(
    lambda x: str(x).startswith('(') or str(x).startswith('{') or 
              str(x).startswith('array') or str(x).startswith('ShapiroResult') or
            str(x).startswith('BartlettResult')
).any()]

df = df.drop(columns=cols_to_drop)

In [120]:
cols_to_drop_2 = [col for col in df.columns if not df[col].apply(lambda x: ',' in str(x)).any()]

df = df.drop(columns=cols_to_drop_2)

In [121]:
cols_to_drop_3 = [col for col in df.columns if df[col].apply(lambda x: 'alternative' in str(x)).any()]

df = df.drop(columns=cols_to_drop_3)

In [122]:
df

,Column_28,Column_44,Column_58,Column_71,Column_84,Column_96,Column_109,Column_122,Column_135,Column_148,...,Column_630,Column_643,Column_656,Column_669,Column_682,Column_695,Column_708,Column_721,Column_734,Column_747
0,"0.8188860205801154, 0.8227465849198665, 0.8261...","0.8264413251586533, 0.8213653149761572, 0.8227...","0.8221890574020292, 0.8238015847405974, 0.8192...","0.8218502384281667, 0.8206975368398408, 0.8203...","0.8191414793302498, 0.821087895736976, 0.82117...","0.8227058011544943, 0.8204680721379656, 0.8270...","0.8207078448244953, 0.8243931734251192, 0.8247...","0.8268523000250977, 0.8226327489154207, 0.8212...","0.8207056039582662, 0.8203044889032304, 0.8175...","0.8204048797103008, 0.8228169481194652, 0.8233...",...,"0.8319103832777598, 0.8313828833673945, 0.8293...","0.8296748951274605, 0.8277360976659137, 0.8210...","0.8316885375210641, 0.833202914918791, 0.83237...","0.8339513642393603, 0.8325328959162452, 0.8299...","0.8343950557527517, 0.8237146391309025, 0.8307...","0.8294391560001433, 0.8325745760281095, 0.8343...","0.8325499264995877, 0.8290358000788784, 0.8328...","0.8320650030475781, 0.8277697106593526, 0.8347...","0.8177248037001184, 0.8166447061776201, 0.8134...","0.7779256749489082, 0.7774739163170915, 0.7775..."


In [123]:
all_data = []

for column in df.columns:
    column_data = list(map(float, df[column].iloc[0].split(', ')))
    all_data.append(column_data)

all_data_array = np.array(all_data).T
new_df = pd.DataFrame(all_data_array, columns=[f'dt_{i}' for i in range(len(df.columns))])

In [124]:
len_all_data = []
len_all_data = [len(sublist) for sublist in all_data]
sum(len_all_data) # 2000 * 56

112000

In [125]:
new_df.shape

(2000, 56)

In [126]:
new_df = new_df.astype(float)

In [127]:
new_df

,dt_0,dt_1,dt_2,dt_3,dt_4,dt_5,dt_6,dt_7,dt_8,dt_9,...,dt_46,dt_47,dt_48,dt_49,dt_50,dt_51,dt_52,dt_53,dt_54,dt_55
0,0.8188860205801154,0.8264413251586533,0.8221890574020292,0.8218502384281667,0.8191414793302498,0.8227058011544943,0.8207078448244953,0.8268523000250977,0.8207056039582662,0.8204048797103008,...,0.8319103832777598,0.8296748951274605,0.8316885375210641,0.8339513642393603,0.8343950557527517,0.8294391560001433,0.8325499264995877,0.8320650030475781,0.8177248037001184,0.7779256749489082
1,0.8227465849198665,0.8213653149761572,0.8238015847405974,0.8206975368398408,0.8210878957369760,0.8204680721379656,0.8243931734251192,0.8226327489154207,0.8203044889032304,0.8228169481194652,...,0.8313828833673945,0.8277360976659137,0.8332029149187909,0.8325328959162452,0.8237146391309025,0.8325745760281095,0.8290358000788784,0.8277697106593526,0.8166447061776201,0.7774739163170915
2,0.8261930371804524,0.8227842314725180,0.8192848947689220,0.8203349646839482,0.8211770822129003,0.8270414291348465,0.8247808432827795,0.8212882291778710,0.8175778028754795,0.8233897135276613,...,0.8293817898246746,0.8210614535154709,0.8323711053744935,0.8299361801297909,0.8307594743824172,0.8343314151518411,0.8328775411423039,0.8347589724283819,0.8134622279588397,0.7775545875013445
3,0.8245643756050338,0.8230383457029149,0.8220546054282745,0.8200812986267972,0.8212057653006346,0.8230540317665196,0.8221254168011185,0.8230876447599584,0.8224646839482270,0.8222934817683124,...,0.8291868344627300,0.8288892474274855,0.8349436198056721,0.8325902620917143,0.8304636800401564,0.8334435839518124,0.8269531390054139,0.8306097845183034,0.8170677817217022,0.7786364777168262
4,0.8197572693700477,0.8215176938797462,0.8260460363558136,0.8206043168047040,0.8203918826861711,0.8219820013624465,0.8238943566024883,0.8247768097235666,0.8217556738732924,0.8225256355096626,...,0.8283823634864294,0.8178722526980029,0.8369716037431428,0.8320354236133521,0.8260388655838801,0.8288722168441431,0.8317620379333835,0.8295256534365926,0.8181111290380410,0.7785181599799218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.8246100892761106,0.8222343228998601,0.8279046108063534,0.8222661432003154,0.8205151303287799,0.8230347603169481,0.8255543903051163,0.8239736832670037,0.8188663009572981,0.8265228926893979,...,0.8264363952529490,0.8328134523681474,0.8284885805456956,0.8247427485568822,0.8331719909648274,0.8314151518410957,0.8299765157219174,0.8280314438349288,0.8181039582661074,0.7782904879710301
1996,0.8209404467390915,0.8229572263454161,0.8225735900469685,0.8235510558961672,0.8250470581908143,0.8219923093471012,0.8218399304435123,0.8170261016098384,0.8222343228998601,0.8215481696604640,...,0.8307415474525832,0.8325082463877236,0.8291352945394571,0.8334068337456528,0.8313640600910688,0.8330680147717900,0.8293566921229071,0.8307052454196694,0.8147605858520669,0.7762961170269980
1997,0.8161588863791188,0.8182823312179557,0.8213393209278980,0.8237863468502384,0.8231208095801512,0.8209601663619087,0.8228362195690366,0.8250824638772365,0.8204210139471514,0.8226390233408626,...,0.8300692875838085,0.8280130687318490,0.8383945537987165,0.8386029543580367,0.8323486967122010,0.8290080133376357,0.8304695062923524,0.8281851672582553,0.8176100713491807,0.7786364777168262
1998,0.8201382166290201,0.8224763364526192,0.8231759348893909,0.8237706607866336,0.8244765336488473,0.8218618909325589,0.8205500878419562,0.8237984475278765,0.8221787494173748,0.8272484851744289,...,0.8272336954573160,0.8342978021584023,0.8278508300168513,0.8291379835789323,0.8302642429457531,0.8290945107740849,0.8290963034670682,0.8312488795668852,0.8171269405901544,0.7785181599799218


In [128]:
driver.close()
driver.quit()

In [129]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 8)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
dt_0: [0.81700803 0.82688726]
dt_1: [0.81684459 0.82694605]
dt_2: [0.81696841 0.8268523]
dt_3: [0.81686911 0.82693849]
dt_4: [0.81678969 0.82693845]
dt_5: [0.81691336 0.82694918]
dt_6: [0.81683494 0.82689537]
dt_7: [0.81702564 0.82694418]
dt_8: [0.81685845 0.8268523]
dt_9: [0.81716592 0.82700442]
dt_10: [0.81701288 0.82684827]
dt_11: [0.81686786 0.82678873]
dt_12: [0.81671528 0.82688746]
dt_13: [0.81694693 0.82693889]
dt_14: [0.8166173 0.82679575]
dt_15: [0.81693681 0.82689633]
dt_16: [0.81693619 0.82694418]
dt_17: [0.81707979 0.82689149]
dt_18: [0.81706313 0.82685857]
dt_19: [0.81688506 0.82697159]
dt_20: [0.81693774 0.82696751]
dt_21: [0.81687545 0.82694439]
dt_22: [0.81750966 0.82694418]
dt_23: [0.817649 0.82692406]
dt_24: [0.81762719 0.82697876]
dt_25: [0.81782869 0.82709073]
dt_26: [0.81757677 0.82700879]
dt_27: [0.81715138 0.82694967]
dt_28: [0.81767009 0.82701383]
dt_29: [0.82068139 0.83519373]
dt_30: [0.82046165 0.83541813]
dt_31: [0.82096637 0.83515562]
dt_32: [0.820

In [130]:
print('AUC 신뢰 구간')
for column in new_df.columns:
    percentiles = np.percentile(new_df[column], [2.5, 97.5])
    rounded_percentiles = np.round(percentiles, 2)
    print(f"{column}: [{rounded_percentiles[0]} {rounded_percentiles[1]}]")

AUC 신뢰 구간
dt_0: [0.82 0.83]
dt_1: [0.82 0.83]
dt_2: [0.82 0.83]
dt_3: [0.82 0.83]
dt_4: [0.82 0.83]
dt_5: [0.82 0.83]
dt_6: [0.82 0.83]
dt_7: [0.82 0.83]
dt_8: [0.82 0.83]
dt_9: [0.82 0.83]
dt_10: [0.82 0.83]
dt_11: [0.82 0.83]
dt_12: [0.82 0.83]
dt_13: [0.82 0.83]
dt_14: [0.82 0.83]
dt_15: [0.82 0.83]
dt_16: [0.82 0.83]
dt_17: [0.82 0.83]
dt_18: [0.82 0.83]
dt_19: [0.82 0.83]
dt_20: [0.82 0.83]
dt_21: [0.82 0.83]
dt_22: [0.82 0.83]
dt_23: [0.82 0.83]
dt_24: [0.82 0.83]
dt_25: [0.82 0.83]
dt_26: [0.82 0.83]
dt_27: [0.82 0.83]
dt_28: [0.82 0.83]
dt_29: [0.82 0.84]
dt_30: [0.82 0.84]
dt_31: [0.82 0.84]
dt_32: [0.82 0.84]
dt_33: [0.82 0.84]
dt_34: [0.82 0.84]
dt_35: [0.82 0.84]
dt_36: [0.82 0.84]
dt_37: [0.82 0.84]
dt_38: [0.82 0.84]
dt_39: [0.82 0.84]
dt_40: [0.82 0.84]
dt_41: [0.82 0.84]
dt_42: [0.82 0.84]
dt_43: [0.82 0.84]
dt_44: [0.82 0.84]
dt_45: [0.82 0.84]
dt_46: [0.82 0.84]
dt_47: [0.82 0.84]
dt_48: [0.82 0.84]
dt_49: [0.82 0.84]
dt_50: [0.83 0.84]
dt_51: [0.83 0.84]
dt_52: [0.83

In [131]:
i = 0
while i < 55:  
    base_column = f'dt_{i}'
    continue_search = True  

    for j in range(i + 1, 56):
        if not continue_search:
            break  
        compare_column = f'dt_{j}'
        base_stats = sms.DescrStatsW(new_df[base_column])
        compare_stats = sms.DescrStatsW(new_df[compare_column])
        cm = sms.CompareMeans(base_stats, compare_stats)
        confint = cm.tconfint_diff(usevar='unequal')

        print(f'{base_column} and {compare_column}:')
        print(confint)
        print(cm.summary())
        print('-------------------------------------------')
        
        if confint[0] <= 0 <= confint[1] or (confint[0] < 0 and confint[1] < 0):
            continue_search = False  
        if compare_column == 'dt_55':
            break
    if compare_column == 'dt_55' and continue_search:
        break
    i += 1  

dt_0 and dt_1:
(-5.005671708309059e-05, 0.00028796589925668073)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1      0.0001   8.62e-05      1.380      0.168   -5.01e-05       0.000
-------------------------------------------
dt_1 and dt_2:
(-0.0001819481436614732, 0.00015841501469506008)
                          Test for equality of means                          
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
subset #1  -1.177e-05   8.68e-05     -0.136      0.892      -0.000       0.000
-------------------------------------------
dt_2 and dt_3:
(-0.00010444573943102893, 0.00023550728258104177)
                          Test for equality of means                          
        

In [112]:
## dt = 49